In [1]:
#!pip install geopy

In [12]:
#!pip install osmnx
#!pip install geopandas
#!pip install pandas
#!pip install openpyxl

In [1]:
#!pip install osmnx
import osmnx as ox
import geopandas as gpd
import pandas as pd

In [2]:
#Einlesen der Exceldatei
hospitals_df=pd.read_excel('data/hospital_data_3.xlsx') 

In [3]:
#auf sächsische Einrichtungen filtern
hospitals_df= hospitals_df[hospitals_df['Land'] == 14].reset_index(drop=True)

In [4]:
# Einheitliche Konvertierung zu String und Entfernen unsichtbarer Zeichen
hospitals_df.columns = hospitals_df.columns.astype(str)
hospitals_df.columns = hospitals_df.columns.str.strip()

new_columns = {
	'Land':'Land',
	'Kreis':'Kreis',
	'Gemeinde':'Gemeinde',
	'Adresse_Name':'Name_Einrichtung',
	'Adresse_Strasse_Standort':'Adresse_Strasse_Standort',
	'Adresse_Haus-Nr._Standort':'Adresse_Haus-Nr._Standort',
	'Adresse_Ort_Standort':'Adresse_Ort_Standort',
    'Traeger':'Traeger',
	'EinrichtungsTyp':'EinrichtungsTyp',
	'Allgemeine_Notfallversorgung':'Allgemeine_Notfallversorgung',
	'INSG':'INSG',
	'0100':'Betten innere Medizin',
    '0300':'Betten Kardiologie ',
    '1500':'Betten Allgemeine Chirurgie',
	'2100':'Betten Herzchirurgie',
	'2200':'Betten Urologie',
	'2600':'Betten HNO',
	'2900':'Betten Psychatrie'
}
hospitals_df = hospitals_df[list(new_columns.keys())].rename(columns=new_columns)

In [5]:
hospitals_df['Adresse'] = hospitals_df.apply(lambda row: ' '.join(map(str, row[['Adresse_Strasse_Standort', 'Adresse_Haus-Nr._Standort', 'Adresse_Ort_Standort']])), axis=1)

# Alte Spalten löschen
hospitals_df.drop(columns=['Adresse_Strasse_Standort', 'Adresse_Haus-Nr._Standort', 'Adresse_Ort_Standort'], inplace=True)

In [6]:
display(hospitals_df)

,Land,Kreis,Gemeinde,Name_Einrichtung,Traeger,EinrichtungsTyp,Allgemeine_Notfallversorgung,INSG,Betten innere Medizin,Betten Kardiologie,Betten Allgemeine Chirurgie,Betten Herzchirurgie,Betten Urologie,Betten HNO,Betten Psychatrie,Adresse
0,14,511,0,DRK Krankenhaus Chemnitz-Rabenstein,2,2,0,298,56.0,NaN,4.0,NaN,NaN,NaN,NaN,Unritzstr. 23 Chemnitz
1,14,511,0,Klinikum Chemnitz Flemmingstraße/Bürgerstraße,1,2,3,1418,171.0,124.0,106.0,NaN,NaN,40.0,NaN,Flemmingstr. 2 Chemnitz
2,14,511,0,Klinikum Chemnitz Flemmingstraße/Bürgerstraße,1,2,1,212,NaN,NaN,NaN,NaN,NaN,NaN,166.0,Dresdner Str. 178 Chemnitz
3,14,511,0,Zeisigwaldkliniken Bethanien Chemnitz,2,2,1,307,101.0,NaN,30.0,NaN,75.0,NaN,NaN,Zeisigwaldstr. 101 Chemnitz
4,14,521,20,Erzgebirgsklinikum gGmbH - Haus Annaberg,1,2,1,451,NaN,105.0,41.0,NaN,NaN,NaN,101.0,Chemnitzer Str. 15 Annaberg-Buchholz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,14,730,270,Sächsisches Krankenhaus Altscherbitz - Hauptst...,1,2,2,253,NaN,NaN,NaN,NaN,NaN,NaN,169.0,Leipziger Str. 59 Schkeuditz
126,14,730,310,Fachkrankenhaus Hubertusburg gGmbH,1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Husarenpark 5 Torgau
127,14,730,310,"HELIOS-Park-Klinikum, Hauptstandort",3,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Husarenpark 10b Torgau
128,14,730,310,"Kreiskrankenhaus Torgau ""Johann Kentmann""",1,2,1,271,94.0,NaN,73.0,NaN,NaN,5.0,NaN,Christianistr. 1 Torgau


In [8]:
from shapely.geometry import Point

# Iteriere über jede Adresse und füge die Geo-Koordinaten in Spalte2 ein
for index, row in hospitals_df.iterrows():
    adresse = row['Adresse']
    try:
        location_point = ox.geocode(adresse)
        point = Point(location_point[1], location_point[0])  # Point(longitude, latitude)
        hospitals_df.at[index, 'Koordinate'] = point
    except:
        hospitals_df.at[index, 'Koordinate'] = None

In [9]:
display(hospitals_df)

,Land,Kreis,Gemeinde,Name_Einrichtung,Traeger,EinrichtungsTyp,Allgemeine_Notfallversorgung,INSG,Betten innere Medizin,Betten Kardiologie,Betten Allgemeine Chirurgie,Betten Herzchirurgie,Betten Urologie,Betten HNO,Betten Psychatrie,Adresse,Koordinate
0,14,511,0,DRK Krankenhaus Chemnitz-Rabenstein,2,2,0,298,56.0,NaN,4.0,NaN,NaN,NaN,NaN,Unritzstr. 23 Chemnitz,POINT (12.825659 50.8209044)
1,14,511,0,Klinikum Chemnitz Flemmingstraße/Bürgerstraße,1,2,3,1418,171.0,124.0,106.0,NaN,NaN,40.0,NaN,Flemmingstr. 2 Chemnitz,POINT (12.885481427852355 50.8436274)
2,14,511,0,Klinikum Chemnitz Flemmingstraße/Bürgerstraße,1,2,1,212,NaN,NaN,NaN,NaN,NaN,NaN,166.0,Dresdner Str. 178 Chemnitz,POINT (12.955032306650038 50.8508545)
3,14,511,0,Zeisigwaldkliniken Bethanien Chemnitz,2,2,1,307,101.0,NaN,30.0,NaN,75.0,NaN,NaN,Zeisigwaldstr. 101 Chemnitz,POINT (12.9616543 50.8399579)
4,14,521,20,Erzgebirgsklinikum gGmbH - Haus Annaberg,1,2,1,451,NaN,105.0,41.0,NaN,NaN,NaN,101.0,Chemnitzer Str. 15 Annaberg-Buchholz,POINT (13.005564254307965 50.59273005)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,14,730,270,Sächsisches Krankenhaus Altscherbitz - Hauptst...,1,2,2,253,NaN,NaN,NaN,NaN,NaN,NaN,169.0,Leipziger Str. 59 Schkeuditz,POINT (12.252296936251213 51.38724355)
126,14,730,310,Fachkrankenhaus Hubertusburg gGmbH,1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Husarenpark 5 Torgau,POINT (12.9820132 51.5618484)
127,14,730,310,"HELIOS-Park-Klinikum, Hauptstandort",3,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Husarenpark 10b Torgau,POINT (12.9799036 51.5619934)
128,14,730,310,"Kreiskrankenhaus Torgau ""Johann Kentmann""",1,2,1,271,94.0,NaN,73.0,NaN,NaN,5.0,NaN,Christianistr. 1 Torgau,POINT (12.984629 51.5615645)


In [10]:
import geopandas as gpd
from shapely.geometry import Point

# Konvertiere Pandas DataFrame in GeoPandas DataFrame
hospitals_gp_df = gpd.GeoDataFrame(hospitals_df, geometry='Koordinate', crs='EPSG:4326')

print(hospitals_gp_df)

# Save to a GeoJSON file
output_file = "data/hospitals_saxony.geojson"
hospitals_gp_df.to_file(output_file, driver="GeoJSON")

     Land  Kreis  Gemeinde                                   Name_Einrichtung  \
0      14    511         0                DRK Krankenhaus Chemnitz-Rabenstein   
1      14    511         0      Klinikum Chemnitz Flemmingstraße/Bürgerstraße   
2      14    511         0      Klinikum Chemnitz Flemmingstraße/Bürgerstraße   
3      14    511         0              Zeisigwaldkliniken Bethanien Chemnitz   
4      14    521        20           Erzgebirgsklinikum gGmbH - Haus Annaberg   
..    ...    ...       ...                                                ...   
125    14    730       270  Sächsisches Krankenhaus Altscherbitz - Hauptst...   
126    14    730       310                 Fachkrankenhaus Hubertusburg gGmbH   
127    14    730       310                HELIOS-Park-Klinikum, Hauptstandort   
128    14    730       310          Kreiskrankenhaus Torgau "Johann Kentmann"   
129    14    730       330                 Fachkrankenhaus Hubertusburg gGmbH   

     Traeger  EinrichtungsT

In [11]:
# Define the place names with measures
places_with_measures = [
    "Leipzig, Germany, 616093",
    "Bennewitz, Germany, 5101",
    "Böhlen, Germany, 6776",
    "Borna, Germany, 19470",
    "Borsdorf, Germany, 8195",
    "Brandis, Germany, 9676",
    "Colditz, Germany, 8381",
    "Frohburg, Germany, 12305",
    "Grimma, Germany, 28205"
]

# Prepare a list to store the geocoded polygons with measures
polygons = []

# Retrieve the boundary polygons for each place and add measure
for place in places_with_measures:
    parts = place.split(", ")
    place_name = ", ".join(parts[:-1])
    measure = int(parts[-1])
    polygon = ox.geocode_to_gdf(place_name)
    polygon['population'] = measure
    polygons.append(polygon)

# Combine the boundary polygons
combined_polygon = gpd.GeoDataFrame(pd.concat(polygons, ignore_index=True))

# Set the coordinate reference system to WGS84 (EPSG:4326)
combined_polygon.set_crs(epsg=4326, inplace=True)

# Save the combined polygon as a GeoJSON file
output_file = "cities.geojson"
combined_polygon.to_file(output_file, driver="GeoJSON")

print(f"GeoJSON file saved as {output_file}")


GeoJSON file saved as cities.geojson


In [13]:
print(combined_polygon)

                                            geometry  bbox_north  bbox_south  \
0  POLYGON ((12.23665 51.31978, 12.23797 51.31710...   51.448115   51.238170   
1  POLYGON ((12.63941 51.34480, 12.63950 51.34425...   51.401274   51.295220   
2  POLYGON ((12.34781 51.20982, 12.34791 51.20913...   51.247565   51.165165   
3  POLYGON ((12.44609 51.11321, 12.44622 51.11226...   51.180846   51.063974   
4  POLYGON ((12.49888 51.36056, 12.49890 51.36053...   51.375187   51.324156   
5  POLYGON ((12.53135 51.31726, 12.53339 51.31458...   51.351974   51.300051   
6  POLYGON ((12.67475 51.12160, 12.68448 51.11743...   51.181530   51.091252   
7  POLYGON ((12.50114 51.06428, 12.50178 51.06374...   51.143143   50.978870   
8  POLYGON ((12.63978 51.18491, 12.64109 51.18089...   51.312505   51.156491   

   bbox_east  bbox_west   place_id  osm_type  osm_id        lat        lon  \
0  12.542537  12.236652  118921247  relation   62649  51.340632  12.374733   
1  12.758772  12.639413  118765423  relatio

In [4]:
'''
addresses = [
    "Eintrag 1, Brandenburger Tor, Berlin, Germany",
    "Schloss Sanssouci, Potsdam, Germany",
    "Hamburger Chaussee 22, Kiel, Germany"
]

coordinates = []
for address in addresses:
    location_point = ox.geocode(address)
    coordinates.append(location_point)

for address, coord in zip(addresses, coordinates):
    print(f"Coordinates of {address}: {coord}")
'''

Coordinates of Brandenburger Tor, Berlin, Germany: (52.5162699, 13.377703399031432)
Coordinates of Schloss Sanssouci, Potsdam, Germany: (52.40434735, 13.0392968883845)
Coordinates of Hamburger Chaussee 22, Kiel, Germany: (54.3069489, 10.1231422)


In [12]:
from shapely.geometry import Point

# Define the addresses
addresses = [
    "Ev. Diakonissenkrankenhaus Hauptstandort,249,Georg-Schwarz-Str. 49 Leipzig",
    "HELIOS-Park-Klinikum Hauptstandort,670,Strümpellstr. 41,Leipzig",
    "Herzzentrum Leipzig GmbH,440,Strümpellstr. 39,Leipzig",
    "Klinikum St. Georg GmbH,63,Nikolai-Rumjanzew-Str. 100,Leipzig",
    "Klinikum St. Georg GmbH,617,Delitzscher Str. 141,Leipzig",
    "St. Elisabeth Krankenhaus Leipzig gGmbH,442,Biedermannstr.,Leipzig",
    "Universitätsklinikum Leipzig AöR,1421,Liebigstr. 18,Leipzig",
    "Neurologisches Rehabilitationszentrum Leipzig,60,Muldentalweg 1,Bennewitz",
    "Sana-Kliniken Leipziger Land Klinikum Borna,507,Rudolf-Virchow-Str. 2,Borna",
    "Diakoniewerk Zschadraß gemeinnützige GmbH,165,Im Park 15a,Colditz",
    "Muldentalkliniken Standort Wurzen,177,Kleiststr. 5,Grimma",
    "Muldentalkliniken Standort Wurzen,178,Kutusowstr. 70,Wurzen",
    "Sana-Kliniken Leipziger Land Klinikum Borna,59,Pestalozzistr. 9,Zwenkau",
    "MediClin Waldkrankenhaus Bad Düben,125,Gustav-Adolf-Str. 15a,Bad Düben",
    "Kreiskrankenhaus Delitzsch GmbH Klinik Delitzsch,135,Dübener Str. 3-9,Delitzsch",
    "Kreiskrankenhaus Delitzsch GmbH Klinik Delitzsch,135,Wilhelm-Grune-Str. 5-8,Eilenburg",
    "COLLM KLINIK OSCHATZ GmbH,190,Parkstr. 1,Oschatz",
    "Helios Klinik Schkeuditz,150,Leipziger Str. 45,Schkeuditz",
    "Sächsisches Krankenhaus Altscherbitz - Hauptstandort,253,Leipziger Str. 59,Schkeuditz",
    "Kreiskrankenhaus Torgau Johann Kentmann,271,Christianistr. 1,Torgau",
    "Fachkrankenhaus Hubertusburg gGmbH,194,Schloss Hubertusburg 63,Wermsdorf"]

geocoded_data = []

# Geocode each address
for address in addresses:
    parts = address.split(",")

    # Ensure parts contain at least three elements: name, measure, and address
    if len(parts) < 3:
        print(f"Skipping invalid address format: {address}")
        continue
    
    name = parts[0]
    measure = int(parts[1])
    full_address = ", ".join(parts[2:])
    
    try:
        location_point = ox.geocode(full_address)
        point = Point(location_point[1], location_point[0])  # Note: Point(x, y) -> Point(longitude, latitude)
        geocoded_data.append({'name': name, 'Bettenkapazität': measure, 'address': full_address, 'geometry': point})
    except Exception as e:
        print(f"Error geocoding address '{full_address}': {e}")
        continue

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(geocoded_data)

print(gdf)

# Set the coordinate reference system to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Save to a GeoJSON file
output_file = "hospitals.geojson"
gdf.to_file(output_file, driver="GeoJSON")

print(f"GeoJSON file saved as {output_file}")

Error geocoding address 'Schloss Hubertusburg 63, Wermsdorf': Nominatim could not geocode query 'Schloss Hubertusburg 63, Wermsdorf'
                                                 name  Bettenkapazität  \
0            Ev. Diakonissenkrankenhaus Hauptstandort              249   
1                  HELIOS-Park-Klinikum Hauptstandort              670   
2                            Herzzentrum Leipzig GmbH              440   
3                             Klinikum St. Georg GmbH               63   
4                             Klinikum St. Georg GmbH              617   
5             St. Elisabeth Krankenhaus Leipzig gGmbH              442   
6                    Universitätsklinikum Leipzig AöR             1421   
7       Neurologisches Rehabilitationszentrum Leipzig               60   
8         Sana-Kliniken Leipziger Land Klinikum Borna              507   
9           Diakoniewerk Zschadraß gemeinnützige GmbH              165   
10                  Muldentalkliniken Standort Wurzen

In [6]:
print(geocoded_data)

[{'name': 'Eintrag 1', 'geometry': <POINT (13.378 52.516)>}, {'name': 'Eintrag 2', 'geometry': <POINT (13.039 52.404)>}, {'name': 'Klinik 1', 'geometry': <POINT (10.123 54.307)>}]
